# Minibatch (Miniloteamento)

Este modelo é computacionalmente ineficiente e só deve ser utilizado quando não houver capacidade computacional disponível para armazenar em memória toda a massa de dados necessárias para o treinamento do Modelo.

O total de espaço na memória necessário para os inputs, pesos e viés é de aproximadamente 174 megabytes, o que não é muito. É possível treinar esse conjunto de dados na maioria dos CPUs e GPUs.

Mas conjuntos de dados maiores que aparecerão no futuro serão medidos em gigabytes ou até mais. É possível comprar mais memória, mas isso é caro. Uma placa de vídeo (GPU) Titan X com 12GB de memória custa mais de R$4.000,00.

Ao invés disso, para rodar modelos grandes na sua máquina, iremos aprender a usar miniloteamento.

Vamos ver como implementar um miniloteamento no TensorFlow.

In [8]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

n_input = 784  # MNIST data input (img shape: 28*28)
n_classes = 10  # MNIST total classes (0-9 digits)

# Importando os dados MNIST
mnist = input_data.read_data_sets('/tmp/tensorflow/mnist/input_data', one_hot=True)

# As características já foram escalonadas e os dados embaralhados
train_features = mnist.train.images
test_features = mnist.test.images

train_labels = mnist.train.labels.astype(np.float32)
test_labels = mnist.test.labels.astype(np.float32)

# Pesos & viés
weights = tf.Variable(tf.random_normal([n_input, n_classes]))
bias = tf.Variable(tf.random_normal([n_classes]))

PermissionDeniedError: /datasets

## Miniloteamento no TensorFlow
Para usar o miniloteamento, primeiro é necessário __dividir os dados em lotes (batches)__.

Infelizmente, as vezes é impossível dividir os dados em lotes de tamanho exatamente igual. Por exemplo, imagine que você quer criar lotes de tamanho 128 cada de um conjunto de dados com 1000 amostras. Uma vez que 128 não divide 1000 sem deixar resto, você terminará o processo com 7 lotes de 128 amostras e 1 lote com 104 amostras. (7*128 + 1*104 = 1000)

Neste caso, o tamanho das amostras varia, então é importante se aproveitar da função do TensorFlow tf.placeholder() para receber os tamanhos variáveis dos lotes.

Continuando o exemplo, se cada amostra tiver n_input = 784 características e n_classes = 10 possíveis etiquetas, as dimensões de features seriam [None, n_input] e labels seria [None, n_classes].

A dimensão None é um marcador de posição para o tamanho do lote. Na hora de execução, o TensorFlow aceitará qualquer tamanho de lote maior do que 0.

Para:
features é (50000, 400)
labels é (50000, 10)
batch_size é 128

Teremos 391 Lotes e o último Lote terá 80 amostras.

In [ ]:
# Features e Labels
features = tf.placeholder(tf.float32, [None, n_input])
labels = tf.placeholder(tf.float32, [None, n_classes])

In [5]:
from pprint import pprint
import math

def batches(batch_size, features, labels):
    """
    Create batches of features and labels
    :param batch_size: The batch size
    :param features: List of features
    :param labels: List of labels
    :return: Batches of (Features, Labels)
    """
    assert len(features) == len(labels)

    # TODO: Implement batching
    output = []
    sample_size = len(features)
    # Remember that it might be random
    for start_index in range(0, sample_size, batch_size):
        end_index = start_index + batch_size
        batch = [features[start_index:end_index], labels[start_index:end_index]]
        output.append(batch)
    return output


# 4 Samples of features
example_features = [
    ['F11','F12','F13','F14'],
    ['F21','F22','F23','F24'],
    ['F31','F32','F33','F34'],
    ['F41','F42','F43','F44']]
# 4 Samples of labels
example_labels = [
    ['L11','L12'],
    ['L21','L22'],
    ['L31','L32'],
    ['L41','L42']]

# PPrint prints data structures like 2d arrays, so they are easier to read
pprint(batches(3, example_features, example_labels))

[[[['F11', 'F12', 'F13', 'F14'],
   ['F21', 'F22', 'F23', 'F24'],
   ['F31', 'F32', 'F33', 'F34']],
  [['L11', 'L12'], ['L21', 'L22'], ['L31', 'L32']]],
 [[['F41', 'F42', 'F43', 'F44']], [['L41', 'L42']]]]


In [12]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import numpy as np
import math
def batches(batch_size, features, labels):
    """
    Create batches of features and labels
    :param batch_size: The batch size
    :param features: List of features
    :param labels: List of labels
    :return: Batches of (Features, Labels)
    """
    assert len(features) == len(labels)
    outout_batches = []
    
    sample_size = len(features)
    for start_i in range(0, sample_size, batch_size):
        end_i = start_i + batch_size
        batch = [features[start_i:end_i], labels[start_i:end_i]]
        outout_batches.append(batch)
        
    return outout_batches


learning_rate = 0.001
n_input = 784  # MNIST data input (img shape: 28*28)
n_classes = 10  # MNIST total classes (0-9 digits)

# Import MNIST data
mnist = input_data.read_data_sets('/tmp/tensorflow/mnist/input_data', one_hot=True)

# The features are already scaled and the data is shuffled
train_features = mnist.train.images
test_features = mnist.test.images

train_labels = mnist.train.labels.astype(np.float32)
test_labels = mnist.test.labels.astype(np.float32)

# Features and Labels
features = tf.placeholder(tf.float32, [None, n_input])
labels = tf.placeholder(tf.float32, [None, n_classes])

# Weights & bias
weights = tf.Variable(tf.random_normal([n_input, n_classes]))
bias = tf.Variable(tf.random_normal([n_classes]))

# Logits - xW + b
logits = tf.add(tf.matmul(features, weights), bias)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Calculate accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


# TODO: Set batch size
batch_size = 128
assert batch_size is not None, 'You must set the batch size'

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    # TODO: Train optimizer on all batches
    for batch_features, batch_labels in batches(batch_size, train_features, train_labels):
        sess.run(optimizer, feed_dict={features: batch_features, labels: batch_labels})

    # Calculate accuracy for test dataset
    test_accuracy = sess.run(
        accuracy,
        feed_dict={features: test_features, labels: test_labels})

# A precisão é baixa, mas você já sabe que pode treinar no conjunto de dados mais de uma vez. 
# É possível treinar um modelo usando o conjunto de dados múltiplas vezes. 
# Trataremos deste assunto na próxima seção quando falaremos sobre épocas (epochs).
print('Test Accuracy: {}'.format(test_accuracy))


Extracting /tmp/tensorflow/mnist/input_data/train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-labels-idx1-ubyte.gz
Test Accuracy: 0.09229999780654907
